In [45]:
import numpy as np
import pandas as pd
import pymorphy2
from razdel import tokenize

In [107]:
import re
from itertools import chain

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/midzay/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [38]:
PATH ='/home/midzay/Mlerning/DATASET/proj_news_viz/small/'

In [4]:
import string
from nltk.corpus import stopwords

In [26]:
stop_words = stopwords.words('russian')
STOPLIST = set(stop_words)
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”","«","»"]

In [34]:
with open('../../../../nlp/preprocessing/stopwords.txt') as sw:
    text =sw.read().splitlines()
for t in text:
    STOPLIST.add(t)

In [39]:
data = pd.read_csv(PATH+'bivs SSR.csv.bz2')

In [41]:
data.shape

(61056, 7)

In [42]:
morph = pymorphy2.MorphAnalyzer()

In [104]:
def get_lemma(word):
    global lemma_dict
    
    if word in lemma_dict:
        lemma=lemma_dict[word]
    else:
        p=[item.normal_form for item in morph.parse(word) if 'Abbr' not in item.tag]
            
        if len(p)>0:
            lemma = p[0]
        else: lemma=word.lower()
        lemma_dict[word]=lemma
    return lemma

def tokenizeText(text):
    lemmas = []
    tokens = list(_.text for _ in list(tokenize(text)))
    for word in tokens:
        lemmas.append(get_lemma(word))
  
    tokens = lemmas
    tokens = [tok for tok in tokens if tok not in STOPLIST]
    tokens = [tok for tok in tokens if tok not in SYMBOLS]
    tokens = re.sub('[\[\]\/:,\'*?"<>|]', '', str(tokens))
    return tokens

In [105]:
%%time
lemma_dict={}
data['text_lema'] = data.text.apply(tokenizeText)

CPU times: user 5min 11s, sys: 300 ms, total: 5min 12s
Wall time: 5min 12s


In [50]:
data.to_csv(PATH+'lemma/biv_ssr.csv.bz2',compression='bz2', index=False)

In [108]:
X = data.text_lema

In [145]:
def show_topics(vectorizer=vect, lda_model=doc_topic, n_words=15):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [101]:
from sklearn.feature_extraction.text import CountVectorizer

In [172]:
%%time
#vect = CountVectorizer(min_df=20,ngram_range=(1,1))
vect = CountVectorizer(min_df=20,ngram_range=(3,3))
#X_vect_20 = vect.fit_transform(X[:1000])
X_vect_40 = vect.fit_transform(X[:15000])

CPU times: user 13.5 s, sys: 364 ms, total: 13.9 s
Wall time: 13.9 s


In [173]:
X_vect_40.shape

(15000, 1718)

In [116]:
from sklearn.decomposition import LatentDirichletAllocation

In [174]:
lda = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42)

In [175]:
%%time
doc_topic = lda.fit_transform(X_vect_40)

CPU times: user 1min 33s, sys: 462 ms, total: 1min 33s
Wall time: 34.5 s


In [176]:
topic_keywords = show_topics(
    vectorizer=vect, lda_model=lda, n_words=10)

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = [
    'Word ' + str(i) for i in range(df_topic_keywords.shape[1])
]
df_topic_keywords.index = [
    'Topic ' + str(i) for i in range(df_topic_keywords.shape[0])
]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9
Topic 0,блок юлия тимошенко,депутат верховный рад,возбудить уголовный дело,премьер министр украина,президент украина леонид,украина леонид кучма,журналист георгий гонгадзе,уголовный дело факт,премьер министр страна,верховный рад украина
Topic 1,грузия эдуард шеварднадзе,иа новости грузия,агентство итар тасс,президент грузия эдуард,сообщать иа новости,исполнять обязанность президент,суд право человек,доллар тысяча кубометр,пресс служба президент,европейский суд право
Topic 2,министр внутренний дело,президент белоруссия александр,белоруссия александр лукашенко,партия регион виктор,регион виктор янукович,министерство внутренний дело,бют юлий тимошенко,социал демократический партия,лидер партия регион,лидер бют юлий
Topic 3,сообщать риа новость,агентство риа новость,грузия михаил саакашвили,президент грузия михаил,информационный агентство риа,сообщать информационный агентство,передавать агентство риа,риа новость ссылка,риа новость сообщить,информационный агентство интерфакс
Topic 4,сообщать итар тасс,министерство иностранный дело,совет национальный безопасность,президент страна виктор,страна виктор ющенко,национальный безопасность оборона,иностранный дело украина,республика южный осетия,сообщать агентство новости,секретарь совет национальный
Topic 5,грузино осетинский конфликт,сообщать риа новость,российский военный база,зона грузино осетинский,риа новость заявить,передавать агентство интерфакс,россия владимир путин,президент россия владимир,премьер министр грузия,сила поддержание мир
Topic 6,украина виктор ющенко,президент украина виктор,министр иностранный дело,украина виктор янукович,радиостанция эхо москва,грузино абхазский конфликт,премьер министр украина,пост премьер министр,абхазия южный осетия,сообщать корреспондент net
Topic 7,сотрудник правоохранительный орган,сообщить риа новость,00 местный время,00 московский время,риа новость пресс,сообщение пресс служба,правоохранительный орган республика,источник правоохранительный орган,тур президентский выбор,пресс секретарь президент
Topic 8,передавать риа новость,возбудить уголовный дело,сообщать риа новость,уголовный дело статья,сообщать интернет издание,сообщить пресс служба,риа новость ссылка,ссылка пресс служба,сообщать иа regnum,интернет издание газета
Topic 9,служба безопасность украина,сообщать риа новость,центральный избирательный комиссия,передавать риа новость,президент виктор ющенко,внутренний дело украина,сообщать агентство интерфакс,процент голос избиратель,верховный суд украина,агентство новости грузия
